In [0]:
import os
from imutils.video import FPS
import imutils
import time
import cv2
import glob
import time
from google.colab.patches import cv2_imshow

In [2]:
datas_url = {"bicycle":"84b35f0d0e6a1e39eea057d2141542688a51b1bc17bce9c6bf582b9ea744d2b3d705bb2238661481f20edc68e303002977968ac12201a7c0a8c66c2e9ce963bd",
        "bolt":"99d2404e08700fa77acfb72a8f98044f64a03a4021cceb720e7500f72dc1cb919253305e3cdabdfe8bd854555461c5e15076aa1e61d77ef58f1d84a5a3272307",
        "car":"b76378184668a4e329967f1ca0db6acc2c7435b58d026ead923a6b16a6dd697cff0aa06b66cf9e441327eb0b74791d662113a14dd54c68fb185a94882f0e4278",
        "cup":"075115b55ad7d733228e0a1f815b05bcc283719d96ed977d4859658faa6b8f87d3f305e9e04cbb67613f141eeb529c245b8d944b2c052d25c59980ac8e53a169",
        "david":"0b55211166a515fef8a37153589f82ef9deca547810d7034f362b5f254599e0295a406d24db4975a8e78023d145ca936530d296a3fa83f1e574f62cfc7ffbe52",
        "diving":"0e286755a04e82f87aa631c1fe010f53fda1a398d5266f311ef84abe8f941c0741e83fb16e3b17fcbbd6fc2bb1649cc5f31349a79fe842d2b5adf56b2a7a308b",
        "face":"aeb28117d66e1c1c3c3fa3988460e46814a6004016e87c30f9a6907854ca6a4e2c8d135734468bce4c7257cc9c08f1f357ce07c4d634f359cf62122de6d616ae",
        "gymnastics":"8490ac66eed4d46cccf6d203e9401ffc7684cb0f5ec79291cdf7eae3044dd86f9d2899ec4e0b9b579b7f71fba8b5c8238e793f6307548f452158ab98a28d5e82",
        "hand":"4d3110c148463e3bb79b9710485978ab73823235411da2fdf0963f2ecd1dfd2c7ad1ada26c5cbabdd605e0069919a677627b6d533e12516f4ef0cc8753e0fb66",
        "iceskater":"af2465cf2acfa3daacdad14ea300ca73678480ae52ae739aa745fd2294eb33737ce05291c97db07107fb0076623d7c5c7b1ced90641586b7ec7bc2e72b7ebd52",
        "juice":"a5979f5dc2714862b1295dc5c6f5e779d1d7c865b838e6c468773d72ced9b371d64cb80ceae3237177c591a323d885cee5b97f152ca73af96daacdb0a78effb6",
        "jump":"0257cfab4e40bec0d2d7d8107afa80abb4d5894f514937720ec388b95b79e88de274ba40bf0f9a73a9961f639b8aa4cd876e0d0d7d911dc06f16f109690da057",
        "singer":"6cc217bd173f88457ba80caffdaaa7d3edc035eb12592fcf680e3cefe9211640b75960f37b1125cafffff19b4297373e4754b731e24260cac085b8f25148aeb7",
        "sunshade":"ee8a5493f27c87f3fd228728d17e6f94b97cdd8a6d53dd530fef7f91367683d4e34a91d091cbf1ec3be86f923ba4d24efa620af6f5df366ddaae1431ad3204a3",
        "torus":"2e1812a52783b51c0fdef53a85e5d9818dde9c180ec316b5d6ee7ce006699f39e8d13cad094a10e6c563c336965c53b7f8f3f2dee804d4b1313f12680f114be9",
        "woman":"1fc978e34637161516698f8634b6d24ab093fa374a5e83929e06da18db4198aff16892a2509b2201c82021f3eef5e2f702ca274d6debb93af3b6bc248ece4ffc"
      }
for example in ["cup", "iceskater", "woman", "diving"] :
    url = datas_url[example]
    !wget https://data.votchallenge.net/vot2013/dataset/"$example".zip
    !wget -O "$example"_vs.zip https://data.votchallenge.net/sequences/"$url".zip

    !unzip "$example".zip -d $example
    !unzip "$example"_vs.zip -d $example/frames

    !rm "$example".zip
    !rm "$example"_vs.zip


--2020-04-16 11:35:53--  https://data.votchallenge.net/vot2013/dataset/cup.zip
Resolving data.votchallenge.net (data.votchallenge.net)... 194.249.1.179
Connecting to data.votchallenge.net (data.votchallenge.net)|194.249.1.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11680 (11K) [application/zip]
Saving to: ‘cup.zip’

cup.zip             100%[===================>]  11.41K  41.2KB/s    in 0.3s    

2020-04-16 11:35:55 (41.2 KB/s) - ‘cup.zip’ saved [11680/11680]

--2020-04-16 11:35:57--  https://data.votchallenge.net/sequences/075115b55ad7d733228e0a1f815b05bcc283719d96ed977d4859658faa6b8f87d3f305e9e04cbb67613f141eeb529c245b8d944b2c052d25c59980ac8e53a169.zip
Resolving data.votchallenge.net (data.votchallenge.net)... 194.249.1.179
Connecting to data.votchallenge.net (data.votchallenge.net)|194.249.1.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7796846 (7.4M) [application/zip]
Saving to: ‘cup_vs.zip’

cup_vs.zip          100%[

In [3]:
rm *.zip

rm: cannot remove '*.zip': No such file or directory


In [0]:

def bb_iou(box1, box2):
    """
    computes intersection over union (iou) between 2 bounding boxes.
    :param box1: prediction box.
    :param box2: gt box.
    :return: iou.
    """
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[0] + box1[2], box2[0] + box2[2])
    y_bottom = min(box1[1] + box1[3], box2[1] + box2[3])
    # object is lost, no box or no iou between boxes
    if x_right == 0 or y_bottom == 0 or x_right < x_left or y_bottom < y_top:
        return 0.0
    overlap_area = (x_right - x_left) * (y_bottom - y_top)  # intersection
    bb1_area = box1[2] * box1[3]
    bb2_area = box2[2] * box2[3]
    combined_area = bb1_area + bb2_area - overlap_area  # union
    iou = overlap_area / float(combined_area)
    return iou


def evaluate(predictions, ground_truth, iou_cutoff=0.7):
    """
    evaluation method.
    :param predictions: dict of the predictions as tuples.
    :param ground_truth: dict of the gt as tuples.
    :param iou_cutoff: value at which an iou is considered as true positives.
    :return: accuracy and robustness metrics.
    accuracy = ratio of the number of times the object was correctly tracked across all frames.
    robustness = precision of the tracking when the object was correctly tracked.
    """
    assert len(predictions) == len(ground_truth)
    tp = 0
    mean_iou = 0.0
    for i in range(len(ground_truth)):
        prediction = predictions[i]
        gt = ground_truth[i]
        iou = bb_iou(prediction, gt)

        if iou >= iou_cutoff:
            tp += 1
            mean_iou += iou
    return float(tp) / len(ground_truth), float(mean_iou) / float(tp)


def get_frames(folder):
    """
    get the full path to all the frames in the video sequence.
    :param folder: path to the folder containing the frames of the video sequence.
    :return: list of the name of all the frames.
    """
    #folder = folder+"/frames"
    names = os.listdir(folder)
    frames = [os.path.join(folder, n) for n in names if n.endswith('.jpg')]
    frames.sort()
    return frames


def init_tracker(gt):
    """
    get the object location in the first frame.
    :param gt: box location for each frame (output of read_ground_truth).
    :return: location of the object in the first frame.
    """
    return gt[0]


def read_ground_truth(path):
    """
    reads ground-truth and returns it as a numpy array.
    :param path: path to groundtruth.txt file.
    :return: dict of the 4 the coordinates for top-left corner and width/height as tuple.
    """
    ground_truth = {}
    with open(path, 'r') as file:
        for i, line in enumerate(file):
            x, y, width, height = line.split(',')
            x = int(float(x))
            y = int(float(y))
            width = int(float(width))
            height = int(float(height))
            ground_truth[i] = (x, y, width, height)
    return ground_truth


def test_ground_truth(folder, gt):
    """
    use this function to see the ground-truth boxes on the sequence.
    :param folder: path to the folder containing the frames of the video sequence.
    :param gt: box location for each frame (output of read_ground_truth).
    :return: void
    """
    frames = get_frames(folder)

    for i, frame in enumerate(frames):
        box = gt[i]
        frame = cv2.imread(frame, cv2.IMREAD_COLOR)
        cv2.rectangle(frame, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color=(0, 0, 255))

        cv2.imshow('sequence', frame)
        cv2.waitKey(delay=24)


def track(folder, first_box):
    """
    code for your tracker.
    :param folder: path to the folder containing the frames of the video sequence.
    :param first_box: box location of the object in the first frame, used to initialize the tracker.
    :return: dict with an entry for each frame being a tuple (x, y, width, height)
    """
    bboxes = []
    tracker = cv2.TrackerCSRT_create()
    

    #modification des paramètres
    fs = cv2.FileStorage("settings.yaml",cv2.FILE_STORAGE_WRITE) 
    
    fs.write("use_gray", 0)
    fs.write("use_rgb", 1)

    # fs.write("template_size", 200.) 
    # fs.write('number_of_scales', 33)
    # fs.write('scale_sigma_factor', 23.000000000000000e-01)
    # fs.write('scale_model_max_area', 1024.)
    # fs.write('filter_lr', 0.1)
    # fs.write('background_ratio', 1)
    # fs.write('psr_threshold', 3.0000000149011612e-02)

    fs.release()

    fs_settings = cv2.FileStorage("settings.yaml",cv2.FILE_STORAGE_READ)
    tracker = cv2.TrackerCSRT_create()
    tracker.read(fs_settings.root())


    files = glob.glob(folder + '/frames/*')
    files.sort()
    tracker.init(cv2.imread(files[0],cv2.IMREAD_COLOR ), first_box)
    last_bbox = first_box
    nbFrames = len(files)
    nbEchec = 0
    for i in range(nbFrames):
      image = cv2.imread(files[i],cv2.IMREAD_COLOR)
      (success, box) = tracker.update(image)

      if success : 
        (x, y, w, h) = [int(v) for v in box]
        last_bbox = (x, y, w, h)
      else :
        nbEchec += 1
        if i > 1:
          x = bboxes[i-1][0] + (bboxes[i-1][0] - bboxes[i-2][0])
          y = bboxes[i-1][1] + (bboxes[i-1][1] - bboxes[i-2][1])
          w = bboxes[i-1][2]
          h = bboxes[i-1][3]
          last_bbox = (x, y, w, h)
        else :
          x = bboxes[i-1][0]
          y = bboxes[i-1][1]
          w = bboxes[i-1][2]
          h = bboxes[i-1][3]
          last_bbox = (x, y, w, h)
      bboxes.append(last_bbox)

    return bboxes,  1.0- nbEchec/nbFrames

def drawbboxes(folder,bboxes,gt):
    names = os.listdir(folder+"/frames")
    names.sort()
    try:
      os.mkdir(folder+"/preds")
      os.mkdir(folder+"/bboxes")
      os.mkdir(folder+"/gt")
    except :
      pass
    for i, name in enumerate(names):
      frame_path = folder + "/frames/" + name
      box = bboxes[i]
      gt_box = gt[i]
      frame = cv2.imread(frame_path, cv2.IMREAD_COLOR)
      framebboxes = cv2.imread(frame_path, cv2.IMREAD_COLOR)
      framegt = cv2.imread(frame_path, cv2.IMREAD_COLOR)

      cv2.rectangle(frame, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color=(0, 0, 255), thickness = 2)
      cv2.rectangle(frame, (gt_box[0], gt_box[1]), (gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]), color=(0, 255, 0), thickness = 2)

      cv2.rectangle(framebboxes, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color=(0, 0, 255), thickness = 2)

      cv2.rectangle(framegt, (gt_box[0], gt_box[1]), (gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]), color=(0, 255, 0), thickness = 2)

      cv2.imwrite(folder+"/preds/"+ name,frame)
      cv2.imwrite(folder+"/bboxes/"+ name,framebboxes)
      cv2.imwrite(folder+"/gt/"+ name,framegt)

def main(folder, cutoff = 0.5):
    path_gt = folder + '/groundtruth.txt'
    gt = read_ground_truth(path_gt)
    # test_ground_truth(folder, gt)
    first_box = init_tracker(gt)
    start = time.time()
    predictions, successProportion = track(folder, first_box)
    end = time.time()
    fps = float(len(get_frames(folder)))/(end-start)
    accuracy, robustness = evaluate(predictions, gt, cutoff)
    print(example +  " : ")
    print(f'cutoff = {cutoff}, accuracy = {accuracy}, robustness = {robustness}, fps = {fps}, success proportion = {successProportion}')
    drawbboxes(folder,predictions,gt)
    return predictions
    

In [5]:
for cutoff in [0.3]:
  for example in ['cup','iceskater', 'woman', 'diving']:
    preds = main(example,cutoff)

cup : 
cutoff = 0.3, accuracy = 1.0, robustness = 0.8491166265009175, fps = 0.0, success proportion = 1.0
iceskater : 
cutoff = 0.3, accuracy = 1.0, robustness = 0.6122966144435682, fps = 0.0, success proportion = 1.0
woman : 
cutoff = 0.3, accuracy = 0.9346733668341709, robustness = 0.6318276317620067, fps = 0.0, success proportion = 1.0
diving : 
cutoff = 0.3, accuracy = 0.329004329004329, robustness = 0.648457736734774, fps = 0.0, success proportion = 0.8831168831168831


In [0]:
img_array = []

folder = 'iceskater'
for filename in get_frames(folder +'/bboxes'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter(folder + '/video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()